In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [3]:
df = pd.read_csv('insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


Split dataset into training and testing sets

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=25)

Scale the data

In [5]:
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()
X_train_scaled['age'] = X_train_scaled['age']/100
X_test_scaled['age'] = X_test_scaled['age']/100

Creating a tf model

In [6]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid')
])
model.compile(
    optimizer = "adam",
    loss = "binary_crossentropy",
    metrics =["accuracy"]
)
model.fit(X_train_scaled, y_train, epochs=5000, verbose=0)

In [7]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 483ms/step - loss: 0.3800 - accuracy: 1.0000


[0.37995365262031555, 1.0]

In [8]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [9]:
y_predicted = model.predict(X_test_scaled)
y_predicted

1/1 [==============================] - 0s 200ms/step


array([[0.6965557 ],
       [0.39225498],
       [0.16415285],
       [0.4999179 ],
       [0.7147267 ],
       [0.80899143]], dtype=float32)

In [10]:
coef, bias = model.get_weights()

Custom Neural Network

In [11]:
def sigmoid(x): 
    return 1/(1+np.exp(-x))
def prediction_function(age, affordibility):
    wieghted_sum = coef[0]*age + coef[1]*affordibility + bias    
    return sigmoid(wieghted_sum)
def binary_cross_entropy(y_predicted, y_true):
    total_error = 0
    for yp, yt in zip(y_predicted, y_true):
        if yp == 0:
            yp = 1e-15
        elif yp == 1:
            yp = 1 - 1e-15
        total_error += yt*np.log(yp) + (1-yt)*np.log(1-yp)
    bce_loss = total_error/(-len(y_true))
    return(bce_loss)

In [12]:
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold):
    w1 = 1
    w2 = 1
    b = 0
    learning_rate = .1
    for i in range(epochs):
        weighted_sum = w1* age + w2* affordibility + b
        y_predicted = sigmoid(weighted_sum)
        loss = binary_cross_entropy(y_predicted, y_true)
        
        if loss <= loss_threshold:
            print(w1, w2, b, loss)
            break

        w1 = w1 - learning_rate * np.mean(age*(y_predicted - y_true))
        w2 = w2 - learning_rate * np.mean(affordibility*(y_predicted - y_true))
        b = b - learning_rate * np.mean(y_predicted-y_true)  


In [13]:
gradient_descent(X_train_scaled.age, X_train_scaled.affordibility, y_train, 5000, .459)

5.248136578023422 1.4637747123981855 -3.0460292935192985 0.45899121910372215


In [18]:
class my_NeuralNetwork():
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.b = 0

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def predict(self, X):
        weighted_sum = self.w1 * X.age + self.w2 * X.affordibility + self.b
        return sigmoid(weighted_sum)
    
    def fit(self, X, y, epochs, loss_threshold):
        self.w1, self.w2, self.b = self.gradient_descent(X.age, X.affordibility, y, epochs, loss_threshold)
    
    def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold):
        w1 = w2 =1        
        b = 0
        learning_rate = .1
        for i in range(epochs):
            weighted_sum = w1* age + w2* affordibility + b
            y_predicted = sigmoid(weighted_sum)
            loss = binary_cross_entropy(y_predicted, y_true)
            if i % 500== 0:
                print('Epoch: ', str(i), w1, w2, b, loss)
            if loss <= loss_threshold:
                print('Epoch: ', str(i), w1, w2, b, loss)
                break

            w1 = w1 - learning_rate * np.mean(age*(y_predicted - y_true))
            w2 = w2 - learning_rate * np.mean(affordibility*(y_predicted - y_true))
            b = b - learning_rate * np.mean(y_predicted-y_true) 
        return w1, w2, b
    
    def get_weights(self):
        return self.w1, self.w2, self.b

In [19]:
custom_model = my_NeuralNetwork()
custom_model.fit(X_train_scaled, y_train, epochs = 5000, loss_threshold = 0.45)

Epoch:  0 1 1 0 0.7113403233723417
Epoch:  500 2.1862900400760497 1.2899173275134093 -1.651516775709162 0.539128843416425
Epoch:  1000 3.429739930974083 1.4030706335279304 -2.2507842646752496 0.5005591454735495
Epoch:  1500 4.460397289206989 1.4382971585065203 -2.70216096116355 0.4751366958664722
Epoch:  2000 5.312186066843866 1.466085088493145 -3.074182020474781 0.4577962288992785
Epoch:  2302 5.755896569450322 1.4834409858991255 -3.270322834324345 0.44998721546522324


In [20]:
custom_model.predict(X_test_scaled)
# custom_model.get_weights()

2     0.714714
10    0.320643
21    0.145072
11    0.456304
14    0.737595
9     0.848671
dtype: float64

In [21]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 64ms/step


array([[0.6965557 ],
       [0.39225498],
       [0.16415285],
       [0.4999179 ],
       [0.7147267 ],
       [0.80899143]], dtype=float32)